In [ ]:
import sys; sys.path.append('../..')
from open_linkage import open_linkage
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer

l = elastic_rods.RodLinkage('data/AsymmOverhanging2BumpsArchitecturalScale/flat_opt.msh')
driver=l.centralJoint()

mat = elastic_rods.RodMaterial()
#mat.setContour(2000, 0.3, '../../../examples/cross_sections/custom_extrusion_B.obj', stiffAxis=elastic_rods.StiffAxis.D1, simplifyVisualizationMesh = 10)
mat.setContour(2000, 0.3, '../../../examples/cross_sections/190108_section_test_R01.obj', stiffAxis=elastic_rods.StiffAxis.D1, simplifyVisualizationMesh = 10)
l.setMaterial(mat)

l.setPerSegmentRestLength(np.loadtxt('data/AsymmOverhanging2BumpsArchitecturalScale/design_parameters.txt'))

jdo = l.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
fixedVars.append(jdo + 6) # constrain angle at the driving joint
with suppress_stdout(): elastic_rods.compute_equilibrium(l, fixedVars=fixedVars)

view = LinkageViewer(l, width=1024)
view.setCameraParams(((-0.32613526983651236, -1.6251272340294551, 0.8696908362175328),
 (0.1303206918666238, -0.7938697110342012, -0.5939590888048278),
 (0.0, 0.0, 0.0)))
view.show()

In [ ]:
from linkage_utils import writeRodSegments
writeRodSegments(l,'rod_segments_asymmOverhangRender.txt', zeroBasedIndexing=True)
np.savetxt('restlen_asymmOverhangRender.txt',l.getPerSegmentRestLength())
np.savetxt('normals_asymmOverhangRender.txt', np.array([j.normal for j in l.joints()]))
l.writeLinkageDebugData('deployed_asymmOverhangRender.msh')

In [ ]:
from open_linkage import open_linkage
def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.beta = 1e-8
    opts.gradTol = 1e-12
    opts.useIdentityMetric = False
    return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)

In [ ]:
#open_linkage(l, driver, np.deg2rad(5.86) - l.averageJointAngle, 50, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=True);
open_linkage(l, driver, np.deg2rad(93) - l.averageJointAngle, 50, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=True);

In [ ]:
# Open slightly to render non-self-intersecting planar state
# (To be run immediately after (re-)running the first cell)
open_linkage(l, driver, np.deg2rad(9) - l.averageJointAngle, 2, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=True);

In [ ]:
from write_render_files import writeRenderFiles
writeRenderFiles(l, 'TeaserRestSlightlyOpen', 'teaser_rest')